In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [2]:
if torch.cuda.is_available(): 
    device='cuda'
else:
    device='cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [3]:
print(torch.cuda.is_available())

True


In [4]:
learning_rate = 0.0001
training_epochs = 20
batch_size = 100

In [5]:
mnist_train = datasets.MNIST(root='MNIST_data/',
    train=True, transform=transforms.ToTensor(), download=False)
mnist_test = datasets.MNIST(root='MNIST_data/',
    train=False, transform=transforms.ToTensor(), download=False)

In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,
    shuffle=True, drop_last=True)

In [7]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [8]:
model = CNN().to(device)
print('총 parameter 개수 : ', sum(p.numel() for p in model.parameters() if p.requires_grad))
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

총 parameter 개수 :  50186


In [9]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [10]:
X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
Y_test = mnist_test.targets.to(device)

In [14]:
lossSet = []
accSet = []

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch
    
    lossSet.append(avg_cost.item())
    
    model.eval()
    with torch.no_grad():
        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
    accSet.append(accuracy.item())
    
    model.train()

    print('[Epoch: {:>4}] cost = {:.9}, Accuracy = {:.9}'.format(epoch+1, avg_cost, accuracy))

[Epoch:    1] cost = 0.0336535163, Accuracy = 0.970099986
[Epoch:    2] cost = 0.0324314311, Accuracy = 0.970799983
[Epoch:    3] cost = 0.031297002, Accuracy = 0.977099955
[Epoch:    4] cost = 0.0303287692, Accuracy = 0.976300001
[Epoch:    5] cost = 0.0292964447, Accuracy = 0.974299967
[Epoch:    6] cost = 0.0281463917, Accuracy = 0.975999951
[Epoch:    7] cost = 0.0271596424, Accuracy = 0.974899948
[Epoch:    8] cost = 0.0264739171, Accuracy = 0.971399963
[Epoch:    9] cost = 0.0256531294, Accuracy = 0.975999951


KeyboardInterrupt: 

In [13]:
%matplotlib tk

RMSEloss = np.array(list(map(np.sqrt, lossSet)))
xdomain = np.arange(training_epochs)
xdomain += 1

ax1 = plt.subplot(121)
plt.plot(xdomain, RMSEloss, marker='o', color='white')
plt.xlabel("Epochs")
plt.ylabel("RMSELoss")
plt.title("RMSELoss of CNN Model")

ax2 = plt.subplot(122)
plt.plot(xdomain, accSet, marker='o', color='white')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy of CNN Model")

plt.style.use(['dark_background'])
plt.tight_layout(w_pad = 4)
plt.show()